In [1]:
### MOUNT DRIVE .....

In [11]:
## First Import statements
import numpy as np
import pandas as pd
from IPython.display import display
import pickle

from scipy import stats
from xml.dom import minidom

In [2]:
FOLDER_TRAIN = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Data/Training/'
# FILE_TOPICS = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Data/topics.txt'
FILE_TOPICS = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Data/topics-all.txt'

In [4]:
# !ls "$FOLDER_TRAIN"

In [3]:
## Set random seed
RANDOM_STATE = 22
np.random.seed(RANDOM_STATE)

In [48]:
%%time
## Read data
topic_remove = '3d_Printer' ## Remove 3D-printer
list_doc_types = []
with open(FILE_TOPICS, 'r') as fp:
    topic = fp.readline()
    while topic:
        topic = topic.replace("\n", "")
        list_doc_types.append(topic)
        topic = fp.readline()

list_doc_types.remove(topic_remove)
print(list_doc_types)

['Astronomy', 'Coffee', 'Space', 'Anime', 'Biology', 'Cooking', 'Windows_Phone', 'Arduino', 'Chess', 'Law', 'Wood_Working']
CPU times: user 1.6 ms, sys: 0 ns, total: 1.6 ms
Wall time: 3.61 ms


In [49]:
print(list_doc_types)

['Astronomy', 'Coffee', 'Space', 'Anime', 'Biology', 'Cooking', 'Windows_Phone', 'Arduino', 'Chess', 'Law', 'Wood_Working']


In [50]:
def get_train_val_test_data(file_name):
    xmldoc = minidom.parse(file_name)
    xml_list = xmldoc.getElementsByTagName('row') ## tag using 'row'
    
    # print(f"Inside get_train_val_test_data(), len(item_list) = {len(item_list)}")

    item_list = [x.attributes['Body'].value for x in xml_list]

    train_list = item_list[0:500] ## first 500 train
    val_list =  item_list[500:700] ## next 200 val
    test_list = item_list[700:1200] ## next 500 test

    ## delete original list.
    del item_list
    del xmldoc

    ## return the new lists
    return train_list, val_list, test_list

In [14]:
def add_to_dataframe(df_old, to_add):
    df_old = df_old.append(pd.Series(to_add, index=df_old.columns), ignore_index=True)
    return df_old

In [52]:
## Create three dataframes.

## https://www.kite.com/python/answers/how-to-create-an-empty-dataframe-with-column-names-in-python
column_names =["content", "Label"]

df_train = pd.DataFrame(columns = column_names)
df_val = pd.DataFrame(columns = column_names)
df_test = pd.DataFrame(columns = column_names)

In [53]:
%%time
### Populate all topics
def populate_data_frames(df_train, df_val, df_test, list_doc_types):
    for topic in list_doc_types: ## iterating per topic/label
        label = topic ## assign label
        
        ## read using xml package
        file_name = FOLDER_TRAIN + topic + ".xml"
        xmldoc = minidom.parse(file_name)
        xml_list = xmldoc.getElementsByTagName('row') ## tag using 'row'

        ## get train, val, test lists
        train_list, val_list, test_list = get_train_val_test_data(file_name=file_name)

        print(len(train_list), len(val_list), len(test_list))

        ## add to dataframe
        for v1 in train_list:
            df_train = add_to_dataframe(df_old=df_train, to_add=[v1, label])
        for v2 in val_list:
            df_val = add_to_dataframe(df_old=df_val, to_add=[v2, label])
        for v3 in test_list:
            df_test = add_to_dataframe(df_old=df_test, to_add=[v3, label])


        ## delete original list
        del train_list
        del val_list
        del test_list

    ## return dataframes
    return df_train, df_val, df_test

### Call the function
df_train, df_val, df_test = populate_data_frames(df_train=df_train, df_val=df_val, df_test=df_test, list_doc_types=list_doc_types)

print(f"len df_train = {len(df_train)}")
print(f"len df_val = {len(df_val)}")
print(f"len df_test = {len(df_test)}")

500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
500 200 500
len df_train = 5500
len df_val = 2200
len df_test = 5500
CPU times: user 1min 11s, sys: 2.08 s, total: 1min 13s
Wall time: 1min 13s


## Shuffle dataframes

In [54]:
df_train = df_train.sample(frac=1, random_state=RANDOM_STATE)
df_val = df_val.sample(frac=1, random_state=RANDOM_STATE)

## Import libraries

In [1]:
## https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
## https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
## from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [12]:
## https://stackoverflow.com/questions/26693736/nltk-and-stopwords-fail-lookuperror
## https://stackoverflow.com/questions/26570944/resource-utokenizers-punkt-english-pickle-not-found

# nltk.download()
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Preprocess each sentence

In [56]:
## https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
## First lemmatize, and then stem.

def pre_process_stem_sentence(sentence, stop_words, stemmer, lemmatizer):
    ## save in another variable
    text = sentence
    ## remove HTML tags [using soup]
    soup = BeautifulSoup(text)
    text = soup.get_text()
    
    ## remove <a href> type things
    soup = BeautifulSoup(text) ## create soup again.
    for a in soup.findAll('a'):
        a.replaceWithChildren()
    
    text = str(soup) ## reform text


    ## remove unicode.
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r'[-+]?\d+', '', text)
    text = re.sub(r'[^\x00-\x7F]',' ', text)

    ## remove links
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"https\S+", "", text)
    ## text = re.sub(r"www\S+", "", text)

    ## to be safe, remove HTML tags again using regex
    #### https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44  
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    ## convert to small letters.
    text = text.lower()

    ## replace newlines, tabs -> SPACE
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")

    ## {COLON} make <a>:<b> become <a>[space]<b>
    # text = text.replace(": ", ":") # :[space] -> : 
    text = text.replace(":", " ")  # : -> [space]
    
    ## {HYPHEN}
    # text = text.replace("- ", "-")
    text = text.replace("-", " ")

    ## numbers removal    
    text = re.sub(r'[-+]?\d+', '', text)

    ## punctuations removal
    text = text.translate((str.maketrans('','',string.punctuation)))   


    ## [remove anything EXCEPT english letters]
    ## https://stackoverflow.com/questions/6323296/python-remove-anything-that-is-not-a-letter-or-number
    text = re.sub(  "[^a-z ]",              # Anything except 0..9, a..z and A..Z
                    "",                     # replaced with nothing
                    text)                   # in this string   

    ## remove space initially and finally.
    text = text.lstrip()

    ## make double spaces become one space.
    text = re.sub('\s+', ' ', text)

    ## remove stop words (English).
    # print(f"Before stopwords removal, text.len = {len(text)}")

    tokens = word_tokenize(text)
    stop_words_removed_words_list = [t for t in tokens if not t in stop_words]

    # print(f"After tokenize and stopwords, len stop_words_removed_words_list = {len(stop_words_removed_words_list)}")

    ## apply lemmatization.
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stop_words_removed_words_list]
    
    ## apply stemming.
    stemmed_words = [stemmer.stem(word) for word in lemmatized_words]

    ## return the final pre-processed list-of-words.
    return stemmed_words

################ Test on one sentence #######################

## Obtain once.
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

## Preprocess per element of dataframe to test.
text_to_process = df_train['content'].iloc[499]
print(text_to_process)

print("--"*90)

preprocessed_text = pre_process_stem_sentence(sentence=text_to_process, stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer)
print(preprocessed_text)

<h2>You never mentioned your tablet model... or Arduino model</h2>

<p>There are some tablets designed to work with USB flash drives and printers. This is called OTG (on the go.</p>

<p><strong>You need:</strong></p>

<ul>
<li>A tablet supporting OTG</li>
<li>An OTG adapter that fits into your tablets USB slot and is compatible</li>
<li><a href="https://play.google.com/store/apps/details?id=com.primavera.arduino.listener" rel="nofollow">The Arduino Uno Communicator App</a></li>
<li>Arduino Uno (or clone) [Note: It says that it works with Atmega16U2 or Atmega8U2 programmed as a USB-to-serial converter so I would assume that that would cover a few boards more than the Uno.)</li>
</ul>

<p>Another alternitive is to look into <a href="https://www.google.com/search?q=arduino+bluetooth&amp;qscrl=1&amp;tbm=isch&amp;imgil=WyPC4O05Xn4rhM%253A%253Bhttps%253A%252F%252Fencrypted-tbn2.gstatic.com%252Fimages%253Fq%253Dtbn%253AANd9GcSfTtWqIuN9YYdd90OO_tlxsvLUYhV_pl7NfaMhHDzSRq9N6W9rNA%253B1200%253B12

In [16]:
#@title Not using this
## https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
## First lemmatize, and then stem.

# def pre_process_stem_sentence(sentence, stop_words, stemmer, lemmatizer):
#     ## save in another variable
#     text = sentence

#     ## remove HTML tags.
#     clean = re.compile('<.*?>')
#     text = re.sub(clean, '', text)

#     ## Convert to small letters.
#     text = text.lower()

#     # Number Removal
#     text = re.sub(r'[-+]?\d+', '', text)


#     ## remove links
#     text = re.sub(r"https\S+", "", text)
#     text = re.sub(r"www\S+", "", text)
#     text = re.sub(r"http\S+", "", text)


#     text = text.replace("-", " ")
    
#     # Remove punctuations
#     text = text.translate((str.maketrans('','',string.punctuation)))    

    

#     ## remove unicodes.
#     # text = re.sub(r"&nbsp;", " ", text)
#     # text = re.sub(r'[-+]?\d+', '', text)
#     text = re.sub(r'[^\x00-\x7F]',' ', text)
#     # encoded_string = text.encode("ascii", "ignore")
#     # text = encoded_string.decode()



#     # text = re.sub(  "[^a-z ]",              # Anything except 0..9, a..z and A..Z
#     #                 "",                     # replaced with nothing
#     #                 text)                   # in this string   

    

#     ## remove space initially and finally.
#     # text = text.lstrip()

#     ## make double spaces become one space.
#     # text = re.sub('\s+', ' ', text)

#     # Tokenize
#     text = word_tokenize(text)

#     # Remove stopwords
#     # stop_words = set(stopwords.words('english'))
#     text = [word for word in text if not word in stop_words]

#     # Lemmatize tokens
#     text = [lemmatizer.lemmatize(word) for word in text]

#     # Stemming tokens
#     text = [stemmer.stem(word) for word in text]

#     return text

################ Test on one sentence #######################

# ## Obtain once.
# stop_words = set(stopwords.words('english'))
# stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

# ## Preprocess per element of dataframe to test.
# text_to_process = df_train['content'].iloc[499]
# print(text_to_process)

# print("--"*90)

# preprocessed_text = pre_process_stem_sentence(sentence=text_to_process, stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer)
# print(preprocessed_text)

In [57]:
doc = df_val['content'].iloc[499]
print(doc)

print("--"*85)

preprocessed = pre_process_stem_sentence(sentence=doc, stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer)
print(len(preprocessed))
print(preprocessed)

<p>This sounds like it's more a matter of determining the original name / wording used to file the copyright. </p>

<p>Without knowing more of that language, you're left to the typical sleuthing options:</p>

<ul>
<li>names of the company owners / major shareholders as of 15-18 years ago. </li>
<li>other DBAs and holding companies of the company originally presumed to hold the copyright. </li>
<li>brute force search of all categorically-related copyrights in the time range. </li>
</ul>

<p>The advice I would really like to give you is knowledge of how IP attorneys filed video game copyrights in the time span of 1985-2000.  This could reveal any unexpected filing categories that could have been used as part of niche or experimental copyright strategy (at that time) for this kind of IP. </p>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
70
['sound', 'like', 'matte

In [58]:
## Preprocess for each sentence of train-dataframe.
df_train["stemmed_content"] = df_train.apply(lambda row : pre_process_stem_sentence(sentence=row['content'], stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer), axis=1)
df_val["stemmed_content"] = df_val.apply(lambda row : pre_process_stem_sentence(sentence=row['content'], stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer), axis=1)
df_test["stemmed_content"] = df_test.apply(lambda row : pre_process_stem_sentence(sentence=row['content'], stop_words=stop_words, stemmer=stemmer, lemmatizer=lemmatizer), axis=1)

# Form Vocabulary from dataframe_train

In [59]:
def print_first_n_keys_and_vals_dict(dictionary, n=5):
    print(f"Len dictionary = {len(dictionary)}, printing first {n} keys, vals")
    itr = 0
    for key in dictionary:
        print(key, dictionary[key])
        itr += 1
        if itr == n:
            break

In [60]:
%%time
### Also remove one-len words
dictionary_vocab = {} ## empty dict.
for doc in df_train['stemmed_content']:
    # print(f"len doc = {len(doc)}")
    for word in doc:
        if len(word) == 1:
            continue

        len_currently = len(dictionary_vocab) ## add to len. [idx new]

        if word not in dictionary_vocab:
            dictionary_vocab[word] = (0, len_currently)
        else:
            (val, idx) = dictionary_vocab[word]
            dictionary_vocab[word] = (val + 1, idx) ## Maintain the same index.

print(f"len dictionary_vocab = {len(dictionary_vocab)}")

len dictionary_vocab = 17506
CPU times: user 267 ms, sys: 971 µs, total: 268 ms
Wall time: 269 ms


In [21]:
# file_name = "/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Vocab.txt"
# with open(file_name, 'w') as fw:
#     for voc in dictionary_vocab:
#         fw.write(voc)
#         fw.write("\n")

In [22]:
# ## https://www.kite.com/python/answers/how-to-save-a-dictionary-to-a-file-in-python
# vocab_file = '/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/vocab_dict_df_train.pkl'

# a_file = open(vocab_file, "wb")
# pickle.dump(dictionary_vocab, a_file)
# a_file.close()

# # a_file = open(vocab_file, "rb")
# # dictionary_vocab = pickle.load(a_file)
# # print(dictionary_vocab)

## Keep only those docs in train whose lengths are above 3 words i.e.
### length of stemmed content > 3

In [18]:
lens = [len(x) for x in df_train['stemmed_content']]
print(max(lens))
print(min(lens))

1381
0


In [19]:
MIN_WORD_COUNT_TO_REMOVE = 3
# print(df_train[df_train['stemmed_content'].str.len() <= MIN_WORD_COUNT_TO_REMOVE]["stemmed_content"])
idxToRemove = df_train[df_train['stemmed_content'].str.len() <= MIN_WORD_COUNT_TO_REMOVE].index
df_train.drop(idxToRemove , inplace=True)

print(f"After removal, len df_train = {len(df_train)}")

After removal, len df_train = 5255


## Create Hamming Distance Vectors by representing with 0/1


In [20]:
def form_hamming_vector(list_words, vocab_dict):
    vec = np.zeros(len(vocab_dict)) # +1 for unknown word.
    for word in list_words:
        if word not in vocab_dict: ## add 1 to unkown word index. [DO NOT]
            # vec[UNKNOWN_WORD_INDEX] = 1
            continue
        else: ## word is present in vocab, get index.
            (value, idx) = vocab_dict[word]
            vec[idx] = 1
    return vec

In [21]:
## Create hamming vectors for each col of dataframe
df_train["ham_vector"] = df_train.apply(lambda row : form_hamming_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)
df_val["ham_vector"] = df_val.apply(lambda row : form_hamming_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)
df_test["ham_vector"] = df_test.apply(lambda row : form_hamming_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)

## Create eucledian vectors by representing how many times

In [22]:
def form_eucledian_vector(list_words, vocab_dict):
    vec = np.zeros(len(vocab_dict)) # +1 for unknown word.
    
    ## Form a small dictionary to store each word count
    dict_local_vocab = {}
    for word in list_words:
        if word not in dict_local_vocab:
            dict_local_vocab[word] = 1
        else:
            dict_local_vocab[word] = dict_local_vocab[word] + 1

    # print(dict_local_vocab)

    # unknown_word_count = 1
    for word in dict_local_vocab:
        if word not in vocab_dict: ## add 1 to unkown word index.
            continue
            # vec[UNKNOWN_WORD_INDEX] = unknown_word_count
            # unknown_word_count += 1
        else: ## word is present in vocab, get index.
            (value, idx) = vocab_dict[word] 
            vec[idx] = dict_local_vocab[word] ## replace with value of this word instead of 1.

    del dict_local_vocab
    return vec

In [23]:
df_train["euc_vector"] = df_train.apply(lambda row : form_eucledian_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)
df_val["euc_vector"] = df_val.apply(lambda row : form_eucledian_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)
df_test["euc_vector"] = df_test.apply(lambda row : form_eucledian_vector(list_words=row['stemmed_content'], vocab_dict=dictionary_vocab), axis=1)

# Stack each of these vertically to form hamming and eucledian vectors.

In [24]:
vocab_size = len(dictionary_vocab) ## PLUS 1 for <UNKNOWN> word
print(f"vocab_size = {vocab_size}")

num_documents = len(df_train)
print(f"num_documents = {num_documents}")

vocab_size = 17506
num_documents = 5255


In [25]:
%%time
hamming_vectors_2D = np.zeros((num_documents, vocab_size))
print(f"hamming_vectors_2D.shape = {hamming_vectors_2D.shape}")

idx = 0
for ham_vec in df_train['ham_vector'].values:
    hamming_vectors_2D[idx] = ham_vec
    idx += 1

print(hamming_vectors_2D.shape)

hamming_vectors_2D.shape = (5255, 17506)
(5255, 17506)
CPU times: user 340 ms, sys: 570 ms, total: 910 ms
Wall time: 914 ms


In [26]:
%%time
eucledian_vectors_2D = np.zeros((num_documents, vocab_size))
print(f"eucledian_vectors_2D.shape = {eucledian_vectors_2D.shape}")

idx = 0
for vec in df_train['euc_vector'].values:
    eucledian_vectors_2D[idx] = vec
    idx += 1

print(eucledian_vectors_2D.shape)

eucledian_vectors_2D.shape = (5255, 17506)
(5255, 17506)
CPU times: user 247 ms, sys: 454 ms, total: 700 ms
Wall time: 702 ms


In [27]:
# df_train.columns.values
labels = df_train['Label'].values

## For now save these dataframes

In [33]:
# df_train.to_csv("/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/train.csv", index=False)
# df_val.to_csv("/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/val.csv", index=False)
# df_test.to_csv("/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/test.csv", index=False)

## Now, finally create TF-IDF and pickel dump everything.

In [28]:
# %%time
def compute_TF_IDF_all_train_set(hamming_vectors_2D, eucledian_vectors_2D, alpha=0.0001, beta=0.0001):
    num_documents, num_words = eucledian_vectors_2D.shape
    print(f"num_documents, num_words = {num_documents, num_words}")

    TF = np.zeros((num_documents, num_words))

    IDF = np.zeros((1, num_words))

    ## Calculate TF(d, w) = N(d, w)/W(d) ; where N(d, w): count(w) in document d , W(d): Total #words in document d
    for itr in range(num_documents): ## iterate row-wise
        doc_eucledian = eucledian_vectors_2D[itr]
        total_num_words_doc_eucledian = num_words - np.sum(doc_eucledian == 0)        
        TF[itr] = doc_eucledian/total_num_words_doc_eucledian

        # if itr == 1:
            # print(f"itr = {itr}, doc_euc[itr] = {np.unique(doc_eucledian[itr], return_counts=True)}")
            # print(f"itr = {itr}, TF[itr] = {np.unique(TF[itr], return_counts=True)}")
            # break

    ## Calculate IDF(d, w) = log( (D + alpha)/(C(w) + beta) ) ; C(w) -> Total # docs with word 'w' ; D -> Total # documents
    D = num_documents
    for itr in range(num_words): ## itereate col-wise
        C_w = np.sum(hamming_vectors_2D[:, itr] == 1) ## first calculate C(w)
        
        IDF[:, itr] = np.log( (D + alpha) / (C_w + beta) )

        # break
        
    TF_IDF = TF*IDF
    del TF
    # del IDF
    return TF_IDF, IDF

TF_IDF_whole_corpus, IDF_whole_corpus = compute_TF_IDF_all_train_set(hamming_vectors_2D=hamming_vectors_2D, eucledian_vectors_2D=eucledian_vectors_2D)
print(f"TF_IDF_whole_corpus.shape = {TF_IDF_whole_corpus.shape}")
print(f"IDF_whole_corpus.shape = {IDF_whole_corpus.shape}")

num_documents, num_words = (5255, 17506)
TF_IDF_whole_corpus.shape = (5255, 17506)
IDF_whole_corpus.shape = (1, 17506)


In [29]:
def form_TF_IDF_for_val_test(list_words, IDF, hamming_vectors_2D, eucledian_vectors_2D, vocab_dict, alpha=0.0001, beta=0.0001):
    num_docs_train, num_words = eucledian_vectors_2D.shape

    TF = np.zeros((1, num_words))
    
    euc_vec = form_eucledian_vector(list_words=list_words, vocab_dict=vocab_dict)
    
    W_d_num_words_in_document = 0
    for word in list_words:
        if word in vocab_dict:
            W_d_num_words_in_document += 1

    TF = euc_vec/W_d_num_words_in_document

    TF_IDF = TF*IDF

    return TF_IDF

words_val = df_val['stemmed_content'].iloc[0]

TF_IDF = form_TF_IDF_for_val_test(list_words=words_val, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
                    eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001)
print(f"TF_IDF.shape = {TF_IDF.shape}")
# print(np.unique(TF_IDF, return_counts=True))

TF_IDF.shape = (1, 17506)


# Now we start with K-Nearest Neighbor

In [30]:
df_val.columns.values

array(['content', 'Label', 'stemmed_content', 'ham_vector', 'euc_vector'],
      dtype=object)

In [31]:
val_ham_0 = df_val['ham_vector'].iloc[0]
print(val_ham_0.shape)

val_euc_0 = df_val['euc_vector'].iloc[0]
print(val_euc_0.shape)

(17506,)
(17506,)


In [32]:
## Similarity functions.
def ham(a, b):
    return np.count_nonzero((a!=b), axis=1)

def euclidean(a, b):
    return np.linalg.norm((a-b), axis=1)

### https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists
def cosine_similarity(a, b):
    cos_sim = np.inner(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return cos_sim

In [40]:
# # a = np.array([x for x in range(40)])
# a = np.array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1])
# a = a.reshape(5, -1)
# print(a)

# b = np.array([1, 0, 1])
# b = b.reshape(1, -1)
# print("\n", b)


# print(ham(a, b))
# print(euclidean(a, b))

# a = np.array([2, 1, 3, 4, 5, 1, 10, 3, 22])
# n = 4
# indices_top = (-a).argsort()[:n]
# print(f"indices_top = {indices_top}")
# print(a[indices_top])

In [44]:
class KNN:
    def __init__(self, K = 1, mode="hamming", to_print=False):
        self.mode = mode
        self.K = K
        if to_print == True:
            print(f"KNN __init__(K={K}, mode={mode})")
        

    def populate_vectors(self, vectors_corpus, labels):
        self.vectors_corpus = vectors_corpus
        self.labels = labels

    def compute_distances(self, v):
        if self.mode == "hamming":
            v = v.reshape(1, -1) ## Reshape vector.
            self.distances = ham(v, self.vectors_corpus)
        elif self.mode == "euclidean":
            v = v.reshape(1, -1) ## Reshape vector.
            self.distances = euclidean(v, self.vectors_corpus)
        elif self.mode == 'cosine_similarity':
            v = v.reshape(-1)
            self.distances = cosine_similarity(v, self.vectors_corpus)

        # print(f"After compute_distances mode={self.mode}, distances = {self.distances}")

    def predict(self, v):
        ## compute distances by using suitable similarity function.
        self.compute_distances(v)

        ## take argmax top results indices
        if self.mode == 'cosine_similarity':
            indices_top = (-self.distances).argsort()[:self.K] ## for some reason, this works for cosine-similarity
        else:
            indices_top = (self.distances).argsort()[:self.K] ## THIS works for hamming and euclidean

        ## apply indices to labels
        top_labels = self.labels[indices_top]

        ## take majority vote and return the label
        top_most_label = stats.mode(top_labels)[0][0]

        ## return the max label
        return top_most_label

################################################# Test #################################################

knn = KNN(K=1, mode='hamming')
knn.populate_vectors(hamming_vectors_2D, labels)
knn.predict(val_ham_0)

# knn = KNN(K=7, mode='euclidean')
# knn.populate_vectors(eucledian_vectors_2D, labels)
# knn.predict(val_euc_0)

'Coffee'

In [34]:
knn = KNN(K=1, mode='cosine_similarity')
knn.populate_vectors(TF_IDF_whole_corpus, labels)
words_val = df_val['stemmed_content'].iloc[0]
tf_idf_val_0 = form_TF_IDF_for_val_test(list_words=words_val, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
                    eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001)
knn.predict(tf_idf_val_0)
tf_idf_val_0 = tf_idf_val_0.reshape(-1)
cs = cosine_similarity(tf_idf_val_0, knn.vectors_corpus)
print(cs.shape)
print(cs)

indices_top = (-cs).argsort()[:knn.K]
print(indices_top)
print(knn.labels[indices_top])

(5255,)
[0.         0.00026631 0.00062275 ... 0.00010283 0.         0.        ]
[3056]
['Wood_Working']


In [35]:
tf_idf_val_0 = tf_idf_val_0.reshape(-1)
print(tf_idf_val_0.shape)
print(TF_IDF_whole_corpus.shape)

cs = cosine_similarity(tf_idf_val_0, TF_IDF_whole_corpus)

print(cs.shape)
print(cs)

(17506,)
(5255, 17506)
(5255,)
[0.         0.00026631 0.00062275 ... 0.00010283 0.         0.        ]


## Applying tests on validation set for KNN

In [62]:
# def add_to_dataframe_results(df_old, to_add):
#     df_old = df_old.append(pd.Series(to_add, index=df_train.columns), ignore_index=True)
#     return df_old

column_names = ["Similarity-Measure", "K", "Accuracy(%)"]

df_results_knn = pd.DataFrame(columns=column_names)

# df_results_knn = add_to_dataframe(df_old=df_results_knn, to_add=["Hamming", k, 22])

display(df_results_knn.head())

,Similarity-Measure,K,Accuracy(%)


In [38]:
# !pip3 install tqdm
from tqdm import tqdm
import operator
from functools import reduce

In [64]:
%%time
for k in [1, 3, 5]: # [1, 3, 5]:
    knn_ham = KNN(K=k, mode='hamming')
    knn_euc = KNN(K=k, mode='euclidean')
    knn_cosine = KNN(K=k, mode='cosine_similarity')


    knn_ham.populate_vectors(hamming_vectors_2D, labels)
    knn_euc.populate_vectors(eucledian_vectors_2D, labels)
    knn_cosine.populate_vectors(TF_IDF_whole_corpus, labels)

    

    ham_correct = 0
    euc_correct = 0
    cosine_correct = 0

    itr = 0
    for (ham_val, lab_true) in (zip(df_val['ham_vector'].values, df_val['Label'].values)):
        itr += 1
        # if itr%300 == 0:
            # print(f"Ham itr = {itr}")
        lab_pred = knn_ham.predict(ham_val)
        if lab_pred == lab_true:
            ham_correct += 1

    itr = 0
    for (euc_val, lab_true) in (zip(df_val['euc_vector'].values, df_val['Label'].values)):
        itr += 1
        # if itr%300 == 0:
            # print(f"Euc itr = {itr}")
        lab_pred = knn_euc.predict(euc_val)
        if lab_pred == lab_true:
            euc_correct += 1

    itr = 0
    for (words_val, lab_true) in (zip(df_val['stemmed_content'].values, df_val['Label'].values)):
        itr += 1
        # if itr%300 == 0:
        #     print(f"TF-IDF itr = {itr}")
        val_tf_idf = form_TF_IDF_for_val_test(list_words=words_val, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
                eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001)
        # if (itr % 10) == 0:
            # print(np.unique(val_tf_idf, return_counts=True))
            # break
        lab_pred = knn_cosine.predict(val_tf_idf)
        # print(knn_cosine.distances.shape)

        if lab_pred == lab_true:
            cosine_correct += 1

    del knn_ham
    del knn_euc
    del knn_cosine


    df_results_knn = add_to_dataframe(df_old=df_results_knn, to_add=["Hamming", k, ham_correct/len(df_val)*100])
    df_results_knn = add_to_dataframe(df_old=df_results_knn, to_add=["Euclidean", k, euc_correct/len(df_val)*100])
    df_results_knn = add_to_dataframe(df_old=df_results_knn, to_add=["TF-IDF-cosine-sim", k, cosine_correct/len(df_val)*100])

    print(f"k = {k}, Hamming: {(ham_correct/len(df_val)*100)}%, Euclidean: {(euc_correct/len(df_val)*100)}%, TF-IDF: {(cosine_correct/len(df_val)*100)}%")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


k = 1, Hamming: 72.16666666666667%, Euclidean: 78.5%, TF-IDF: 91.66666666666666%
k = 3, Hamming: 78.33333333333333%, Euclidean: 80.33333333333333%, TF-IDF: 94.0%
k = 5, Hamming: 78.33333333333333%, Euclidean: 81.0%, TF-IDF: 95.33333333333334%
CPU times: user 4min 9s, sys: 4.49 s, total: 4min 14s
Wall time: 3min 32s


In [47]:
print(f"len df_results_knn = {len(df_results_knn)}")
display(df_results_knn)
# df_results_knn.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/KNN-val-3-topics.csv', index=False)

len df_results_knn = 9


,Similarity-Measure,K,Accuracy(%)
0,Hamming,1,46.454545
1,Euclidean,1,58.090909
2,TF-IDF-cosine-sim,1,72.090909
3,Hamming,3,45.954545
4,Euclidean,3,55.909091
5,TF-IDF-cosine-sim,3,76.954545
6,Hamming,5,48.090909
7,Euclidean,5,57.272727
8,TF-IDF-cosine-sim,5,78.727273


In [42]:
mat = df_results_knn.values
print(mat.shape)
print(mat)

## Naive Bayes

### Combine all documents of each class i into one document

In [49]:
# unique_labels = np.unique(df_train['Label'].values)
# dictionary_list_words_for_NB = {}
# # for label in unique_labels:
# #     if label not in dictionary_list_words:
# #         dictionary_list_words_for_NB[label] = [] ## create new list.
    
# for (list_words, label) in zip(df_train['stemmed_content'].values, df_train['Label'].values):
#     if label not in dictionary_list_words_for_NB:
#         dictionary_list_words_for_NB[label] = [] ## create new list.
#     dictionary_list_words_for_NB[label].append(list_words)

# ## reduce/flat-out to make 1D list.
# ## https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
# for label in dictionary_list_words_for_NB:
#     dictionary_list_words_for_NB[label] = reduce(operator.concat, dictionary_list_words_for_NB[label])

In [36]:
## https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
def keywithmaxval(d):
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

In [39]:
class NaiveBayes:
    def __init__(self, vocab_size, alpha=0.01, to_print=False):
        if to_print == True:
            print(f"NaiveBayes __init(alpha={alpha})__")
        self.alpha = alpha
        self.vocab_size = vocab_size
        self.dictionary_list_words_for_NB = {}
        self.dictionary_prior_probabilities = {}
        self.dictionary_count_words_per_class = {}
        self.dictionary_total_words_per_class = {}
    

    def fit(self, list_list_words, labels):
        self.list_list_words = list_list_words
        self.labels = labels
        self.form_dictionary_list_words()
        self.compute_probabilities()


    def form_dictionary_list_words(self):
        for (list_words, label) in zip(self.list_list_words, self.labels):
            if label not in self.dictionary_list_words_for_NB:
                self.dictionary_list_words_for_NB[label] = [] ## create new list.
            self.dictionary_list_words_for_NB[label].append(list_words)
        
        for label in self.dictionary_list_words_for_NB: ## reduce/flat-out to make 1D list.
            self.dictionary_list_words_for_NB[label] = reduce(operator.concat, self.dictionary_list_words_for_NB[label])


    def compute_probabilities(self):
        ## Compute prior probabilities/initial guesses
        (classes, cnts) = np.unique(self.labels, return_counts=True)
        cnts = cnts/np.sum(cnts) ## C_i / (C_1 + C_2 + ... + C_n)
        for (lab, itr) in zip(classes, range(len(classes))):
            self.dictionary_prior_probabilities[lab] = cnts[itr]

        ## Compute per-word probabilities

        ## Counter increment
        for lab in classes:
            num_words_this_class = 0
            self.dictionary_count_words_per_class[lab] = {}
            for word in self.dictionary_list_words_for_NB[lab]:
                if word not in self.dictionary_count_words_per_class[lab]:
                    self.dictionary_count_words_per_class[lab][word] = 0 ## initialize counter to 0.
                self.dictionary_count_words_per_class[lab][word] = self.dictionary_count_words_per_class[lab][word] + 1 ## increment counter
                num_words_this_class += 1
            self.dictionary_total_words_per_class[lab] = num_words_this_class

    def predict(self, list_words):
        ## Compute probabilities for each label.
        dict_probabilities_per_class = {}

        for lab in np.unique(self.labels):
            prob_log_curr_class = np.log(self.dictionary_prior_probabilities[lab]) ## start with prior probabilities
            # prob_log_curr_class = (self.dictionary_prior_probabilities[lab]) ## start with prior probabilities
            # print(f"Before, prob_log_curr_class = {prob_log_curr_class}")
            for word in list_words: ## iterate per word
                if word in self.dictionary_count_words_per_class[lab]: ## if word exists in THIS document.
                    ## use smoothing factor alpha
                    # prob_log_curr_class += np.log((self.dictionary_count_words_per_class[lab][word] + self.alpha)/(self.dictionary_total_words_per_class[lab] + self.alpha*self.vocab_size)) 
                    word_prob = self.dictionary_count_words_per_class[lab][word]
                    # prob_log_curr_class = prob_log_curr_class*word_prob
                else:
                    word_prob = 0
                prob_log_curr_class += np.log( (word_prob + self.alpha)/(self.dictionary_total_words_per_class[lab] + self.alpha*self.vocab_size) )


            dict_probabilities_per_class[lab] = prob_log_curr_class
            # print(dict_probabilities_per_class)

        ## Get max probability class.
        ## https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
        return keywithmaxval(d=dict_probabilities_per_class)




######################################### Checking ############################################################
NB = NaiveBayes(alpha=0.01, vocab_size=len(dictionary_vocab))
NB.fit(list_list_words=df_train['stemmed_content'].values, labels=df_train['Label'].values)

check = df_val['stemmed_content'].iloc[0]
p = NB.predict(check)
print(p)

Wood_Working


In [ ]:
list_new = [
['coffee', 'tea', 'dew', 'dew', 'dew', 'dew'],
['coffee', 'noir', 'homelander', 'dew'],
['noir', 'noir', 'fool', 'fool', 'noir']
]

labs_new = [
    'bev',
    'supe',
    'misc'
]

vocab_size = 6

NB = NaiveBayes(alpha=0.01, vocab_size=6)
NB.fit(list_new, labs_new)
print(NB.predict(['coffee', 'tea', 'dew', 'dew', 'dew', 'dew']))

## Validation on NaiveBayes

In [68]:
column_names = ["ALPHA", "Accuracy(%)"]

df_results_NB = pd.DataFrame(columns=column_names)

display(df_results_NB.head())

,ALPHA,Accuracy(%)


In [69]:
alpha_values = np.linspace(start=0.01, stop=1.0, num=10)
print(alpha_values)

[0.01 0.12 0.23 0.34 0.45 0.56 0.67 0.78 0.89 1.  ]


In [72]:
%%time
### Validation NaiveBayes ###
for alpha in (alpha_values):
    NB = NaiveBayes(alpha=alpha, vocab_size=len(dictionary_vocab), to_print=True)
    NB.fit(list_list_words=df_train['stemmed_content'].values, labels=df_train['Label'].values)

    nb_acc = 0
    ## Predict each val set ##
    for (x, y) in (zip(df_val['stemmed_content'].values, df_val['Label'].values)):
        y_pred = NB.predict(x)
        if y_pred == y:
            nb_acc += 1


    ## Append to dataframe and print.
    # print(f"NB alpha = {alpha}, accuracy = {nb_acc/len(df_val)*100} %")
    df_results_NB = add_to_dataframe(df_old=df_results_NB, to_add=[alpha, nb_acc/len(df_val)*100])
    del NB

NaiveBayes __init(alpha=0.01)__
NaiveBayes __init(alpha=0.12)__
NaiveBayes __init(alpha=0.23)__
NaiveBayes __init(alpha=0.34)__
NaiveBayes __init(alpha=0.45)__
NaiveBayes __init(alpha=0.56)__
NaiveBayes __init(alpha=0.67)__
NaiveBayes __init(alpha=0.78)__
NaiveBayes __init(alpha=0.89)__
NaiveBayes __init(alpha=1.0)__
CPU times: user 9.49 s, sys: 99.4 ms, total: 9.59 s
Wall time: 9.46 s


In [41]:
df_results_NB.sort_values(by=['Accuracy(%)'], inplace=True, ascending=False)
display(df_results_NB.head(10))
# df_results_NB.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/NB-val-3-topics.csv', index=False)

In [73]:
display(df_results_NB.head(10))

,ALPHA,Accuracy(%)
1,0.02,88.500000
3,0.04,88.500000
2,0.03,88.454545
4,0.05,88.454545
6,0.07,88.454545
24,0.25,88.409091
5,0.06,88.409091
8,0.09,88.409091
7,0.08,88.409091
14,0.15,88.409091


# Hypothesis testing on Test Dataset

### Create and fit best performing models on validation set

In [63]:
words_val = df_val['stemmed_content'].iloc[0]
print(words_val)

['sagul', 'give', 'deflect', 'valu', 'horizont', 'shelf', 'span', 'use', 'variou', 'materi', 'thick', 'unless', 'work', 'extrem', 'heavi', 'load', 'weight', 'countertop', 'peopl', 'danc', 'materi', 'vertic', 'cabinet', 'wall', 'adequ', 'make', 'wall', 'thinner', 'still', 'remain', 'surprisingli', 'strong', 'long', 'cabinet', 'design', 'prevent', 'rack', 'even', 'particleboard', 'known', 'strength', 'hold', 'well', 'vertic', 'compressionbuckl', 'may', 'know', 'youv', 'ever', 'pack', 'book', 'store', 'cheap', 'particleboard', 'bookcas']


In [61]:
##### KNN was K=5, TF-IDF #####
knn = KNN(K=5, mode='cosine_similarity', to_print=True)
knn.populate_vectors(TF_IDF_whole_corpus, labels)

KNN __init__(K=5, mode=cosine_similarity)


In [65]:
val_tf_idf = form_TF_IDF_for_val_test(list_words=words_val, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
                eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001)
print(knn.predict(val_tf_idf))

Wood_Working


In [66]:
##### Naive Bayes was alpha = 0.02/0.04, we will take 0.04 #####
NB = NaiveBayes(alpha=0.04, vocab_size=len(dictionary_vocab), to_print=True)
NB.fit(list_list_words=df_train['stemmed_content'].values, labels=df_train['Label'].values)

NaiveBayes __init(alpha=0.04)__


In [69]:
print(NB.predict(words_val))

Wood_Working


In [68]:
del df_train
del df_val

### Split test dataset 50 iterations per 10 of each topic

In [70]:
df_test.sort_values(by=['Label'], inplace=True)
df_test.drop(labels='content', axis=1, inplace=True)

In [117]:
column_names = ["KNN-Acc(%)", "NB-Acc(%)"]

df_results_test_set = pd.DataFrame(columns=column_names)

display(df_results_test_set.head())

,KNN-Acc(%),NB-Acc(%)


In [ ]:
%%time
initial_offset = np.array([i*num_docs_in_each_topic for i in range(0, 11)])
for counter_test in range(0, 50): ## run iterations 50 times
    offsets = initial_offset + counter_test*10
    start_indices = offsets
    end_indices = offsets + 10
    
    # print("\nCounter = ", counter_test)
    # print(offsets)
    # print(start_indices)
    # print(end_indices)

    ### Testing here ###
    nb_correct = knn_correct = 0
    for i in range(len(start_indices)): ## add all to list.
        # print(np.unique(df_test.iloc[start_indices[i]:end_indices[i]]['Label'].values, return_counts=True), end=' ')
        for (x, y) in zip(df_test.iloc[start_indices[i]:end_indices[i]]['stemmed_content'].values, df_test.iloc[start_indices[i]:end_indices[i]]['Label'].values):
            y_pred = NB.predict(x)
            if y_pred == y:
                nb_correct += 1
            x_tf_idf = form_TF_IDF_for_val_test(list_words=x, IDF=IDF_whole_corpus, hamming_vectors_2D=hamming_vectors_2D, 
                eucledian_vectors_2D=eucledian_vectors_2D, vocab_dict=dictionary_vocab, alpha=0.0001, beta=0.0001)
            y_pred = knn.predict(x_tf_idf)
            if y_pred == y:
                knn_correct += 1
    
    print(f"Done for counter_test = {counter_test}")

    NUM_DOCUMENTS = 10* len(np.unique(df_test['Label'].values))
    knn_acc = knn_correct/( NUM_DOCUMENTS )*100  ## 10*11 total 110 test documents per iteration. [50 iterations]
    nb_acc = nb_correct/( NUM_DOCUMENTS )*100
    print(f"KNN-Acc = {knn_acc}%, NB-Acc = {nb_acc}%")

    df_results_test_set = add_to_dataframe(df_old=df_results_test_set, to_add=[knn_acc, nb_acc])



In [119]:
print(f"len df_results_test_set = {len(df_results_test_set)}")
display(df_results_test_set)

len df_results_test_set = 50


,KNN-Acc(%),NB-Acc(%)
0,81.818182,87.272727
1,76.363636,84.545455
2,78.181818,90.909091
3,85.454545,90.000000
4,82.727273,91.818182
5,77.272727,86.363636
6,70.909091,89.090909
7,82.727273,91.818182
8,75.454545,90.909091
9,80.909091,89.090909


In [121]:
df_results_test_set.to_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Test-Set-KNN-NB.csv', index=False)

## Load and analyze.

In [3]:
df_results_test_set = pd.read_csv('/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Assignment-2 Text NaiveBayes KNN/Test-Set-KNN-NB.csv')
display(df_results_test_set.head(5))

,KNN-Acc(%),NB-Acc(%)
0,81.818182,87.272727
1,76.363636,84.545455
2,78.181818,90.909091
3,85.454545,90.000000
4,82.727273,91.818182


In [5]:
knn_acc = df_results_test_set['KNN-Acc(%)'].values
nb_acc = df_results_test_set['NB-Acc(%)'].values
print(knn_acc.shape, nb_acc.shape)

(50,) (50,)


In [19]:
column_names = ["Method", "Mean-Acc(%)", "Minimum-Acc(%)", "Maximum-Acc(%)", "Std-dev", "Std-Error"]

df_stats = pd.DataFrame(columns=column_names)

display(df_stats.head())

,Method,Mean-Acc(%),Minimum-Acc(%),Maximum-Acc(%),Std-dev,Std-Error


In [20]:
## Summarized results.
df_stats = add_to_dataframe(df_old=df_stats, to_add=["KNN K=5, TF-IDF", np.mean(knn_acc), min(knn_acc), max(knn_acc), np.std(knn_acc), stats.sem(knn_acc, axis=None, ddof=0)])
df_stats = add_to_dataframe(df_old=df_stats, to_add=["NB alpha = 0.04", np.mean(nb_acc), min(nb_acc), max(nb_acc), np.std(nb_acc), stats.sem(nb_acc, axis=None, ddof=0)])

In [21]:
display(df_stats)

,Method,Mean-Acc(%),Minimum-Acc(%),Maximum-Acc(%),Std-dev,Std-Error
0,"KNN K=5, TF-IDF",79.454545,69.090909,89.090909,4.189716,0.592515
1,NB alpha = 0.04,89.672727,81.818182,95.454545,2.832865,0.400628


## Computing T-statistics

In [25]:
## https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

# ans = stats.ttest_rel(knn_acc, nb_acc)
ans = stats.ttest_ind(knn_acc, nb_acc, equal_var=True)
print(ans)

Ttest_indResult(statistic=-14.142663823802279, pvalue=2.0995095838767057e-25)


In [ ]:
# Ttest_indResult(statistic=-14.142663823802279, pvalue=3.641774754361954e-24)
# Ttest_indResult(statistic=-14.142663823802279, pvalue=2.0995095838767057e-25)